# Preprocessing

This notebook is used to process the raw transcripts into a format that can then be used for model training and testing.


In [99]:
#!/usr/bin/env/python
%matplotlib inline
import pandas as pd
import numpy as np
import glob
import string
import unicodedata
import matplotlib
import matplotlib.pyplot as plt
import re

In [100]:
#Load Subset 1 and Subset 2
path1 = '../data/Subset 1/'
subdirs = glob.glob('%s*'%(path1))

path2 = '../data/Subset 2/'
subdirs.extend(glob.glob('%s*'%(path2)))                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [101]:
def resolve_label(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    elif label =='8':
        return '5'
    elif label =='9':
        return '6'
    else:
        return 'nan'
    
def resolve_slabel(label):
    if label == '1':
        return '0'
    elif label =='2':
        return '1'
    elif label =='3':
        return '2'
    elif label =='4':
        return '3'
    elif label =='5':
        return '4'
    else:
        return 'nan'

In [102]:
'''
To avoid overfitting the models we carefully divide the training and testing set. 
Testing set include some teachers who particiapted in Year 1. The transcripts from these teachers are not included
in the training set. The teachers in this list were picked at random.
'''

train_subdirs =[]
test_subdirs = []
for item in subdirs:
    if item.find("Travers.Spring") != -1 or item.find("Benson") != -1 \
    or item.find("Carroll") != -1 or item.find("Saunders.Spring") != -1 \
    or item.find("Keene") != -1 or item.find("Carter") != -1 or item.find("Basker") != -1 :
        test_subdirs.append(item)
    else:
        train_subdirs.append(item)
print(len(train_subdirs), len(test_subdirs))

504 63


In [103]:
data = None
temp = []
flag = 0
for dir in subdirs: #replace with test_subdirs to generate test data
    if dir.endswith('.xlsx') and not dir.startswith('~'):
        try:
            temp_data = None
            load_data = pd.read_excel(dir, engine='openpyxl')
            #print(dir, load_data.columns)
            load_data['Sentence'] = load_data['Sentence'].str.replace("[\(\[].*?[\)\]]", "", regex=True)
            load_data = load_data[load_data['Sentence'] != ""]
            mask = load_data['Sentence'].str.contains('(.*[\(].*|.*[\)].*|.*[\[].*|.*[\]].*)|(^[.?!]+$)|(^.*[0-9]{1,2}:[0-9]{1,2}:[0-9]{1,2}.*$)',regex=True,na=False)
            temp_data = load_data[:][~mask]
            temp_data = temp_data.loc[:, ~temp_data.columns.str.contains('^Unnamed')]
            temp_data['Transcript'] = dir.split('/')[len(dir.split('/'))-1]
            for item in temp_data.columns:
                if item not in ['Teacher Tag', 'TimeStamp' ,'Turn','Student Tag','Sentence','Speaker','Transcript']:
                    temp_data = temp_data.drop(columns=[item])
            if flag == 0:
                data = temp_data
                flag = 1
            else:
                if len(data.columns) > 8:
                    print(data.columns,dir)
                data = data.append(temp_data, ignore_index=True,sort=False)
        except Exception as e:
            print(e,dir)
            pass 

/home/abhijit/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [104]:
len(data), len(subdirs)

(235967, 567)

In [105]:
dataset = []
bad_sentences = []
bad_speakers = []
num_sentences = []
bad_entries = []
translator = str.maketrans('', '', string.punctuation)
for entry in data.iterrows():
    try:
        if type(entry[1]['Speaker']) == int or entry[1]['Speaker'] == float:
            bad_speakers.append(entry[1]['Speaker'])
            continue
        if repr(type(entry[1]['Sentence'])) == "<type 'datetime.time'>" or repr(type(entry[1]['Sentence'])) == "<type 'datetime.datetime'>" or repr(entry[1]['Speaker']) == 'nan':
            bad_sentences.append(entry[1]['Sentence'])
            continue
    
        if repr(type(entry[1]['Sentence'])) == "<type 'int'>" or repr(type(entry[1]['Sentence'])) == "<type 'float'>":
            num_sentences.append(str(entry[1]['Sentence']))
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
        if repr(type(entry[1]['Sentence'])) == "<type 'str'>":
            entry[1]['Sentence'] = str(entry[1]['Sentence']).translate(translator)
            
        temp = unicodedata.normalize('NFKD', str(entry[1]['Sentence']).translate(translator))
        
        # Turn, Speaker, Sentence, Tag
        try:
            tag = resolve_label(str(entry[1]['Teacher Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        try:
            stag = resolve_slabel(str(entry[1]['Student Tag']).split('-')[0].strip())
        except Exception as e:
            print(e,'####')
        
        sent = ' '.join([re.sub(r"[^a-zA-Z0-9]+", ' ', k) for k in temp.split(" ")])
        
        if str(stag) != 'nan':
            #print([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'S',sent.strip(),tag,stag,entry[1]['Transcript']])
        elif str(tag) != 'nan':
            #print([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
            dataset.append([entry[1]['Turn'],'T',sent.strip(),tag,stag,entry[1]['Transcript']])
       
    except Exception as e:
        print(e)
        bad_entries.append([entry[1]['Sentence'], entry[1]['Speaker']])
        continue

In [106]:
df = pd.DataFrame({'Turn':[i[0] for i in dataset], 'Speaker':[i[1] for i in dataset], 'Sentence':[i[2] for i in dataset], 'Tag':[i[3] for i in dataset],'StudentTag':[i[4] for i in dataset],'Transcript':[i[5] for i in dataset]})
df = df.applymap(lambda x: x.encode('unicode_escape').decode('utf-8') if isinstance(x, str) else x)
df

,Turn,Speaker,Sentence,Tag,StudentTag,Transcript
0,1.0,T,This goes under your desk Jared,0,nan,Math US2 transcript.xlsx
1,2.0,S,Bless you,nan,0,Math US2 transcript.xlsx
2,3.0,T,Okay everybody please put your books away,0,nan,Math US2 transcript.xlsx
3,3.0,T,Quietly take out paper to start the warmup,0,nan,Math US2 transcript.xlsx
4,4.0,T,Quietly have your homework out on your desk,0,nan,Math US2 transcript.xlsx
...,...,...,...,...,...,...
234055,53,T,And we can talk about those as well,0,nan,TalkBack.Year2.Gautier.Spring.020221.xlsx
234056,53,T,Right,0,nan,TalkBack.Year2.Gautier.Spring.020221.xlsx
234057,53,T,All right,0,nan,TalkBack.Year2.Gautier.Spring.020221.xlsx
234058,53,T,Um last thing before we start the quiz make su...,0,nan,TalkBack.Year2.Gautier.Spring.020221.xlsx


In [109]:
df[df['Speaker']=='S']

,Turn,Speaker,Sentence,Tag,StudentTag,Transcript
1,2.0,S,Bless you,nan,0,Math US2 transcript.xlsx
8,7.0,S,Oh sorry,nan,0,Math US2 transcript.xlsx
10,9.0,S,Can I my backpack,nan,0,Math US2 transcript.xlsx
14,14.0,S,Can I borrow a pencil,nan,0,Math US2 transcript.xlsx
32,25.0,S,Can I sharpen my pencil,nan,0,Math US2 transcript.xlsx
...,...,...,...,...,...,...
233991,46,S,Im not sure if thats right But,nan,0,TalkBack.Year2.Gautier.Spring.020221.xlsx
233995,48,S,Yeah,nan,0,TalkBack.Year2.Gautier.Spring.020221.xlsx
233996,48,S,Would that angle be the same as The correspond...,nan,3,TalkBack.Year2.Gautier.Spring.020221.xlsx
234005,50,S,Oh yes,nan,0,TalkBack.Year2.Gautier.Spring.020221.xlsx


In [ ]:
'''
  Number transcripts - 567
  Number of teacher utterances - 165,674
  Number of words (with repetitions) - 1,700,036
  Number of unique words - 15,749
  Number of student utterances - 56,580
  Number of Turns - 87,071
'''

In [110]:
for item in sorted(df['Transcript'].unique()):
    print(item, len(df[df['Transcript']==item]))

7th grade math.xlsx 142
Boats and Fish 1_Grade 4.xlsx 210
Boats and Fish 2_Grade 4.xlsx 112
Boats and Fish 3_Grade 4.xlsx 113
Boats and Fish 4_Grade 4 .xlsx 187
Borko.TranscriptofWholeClassDebrief.xlsx 60
CECILIO DIMAS pt 1.xlsx 87
CECILIO DIMAS pt 2.xlsx 44
CECILIO DIMAS pt 3.xlsx 62
Comparing Fractions 1_Grade 4.xlsx 80
Comparing Fractions 2_Grade 4.xlsx 179
Comparing Fractions 3_Grade 4.xlsx 106
Comparing Fractions 4_Grade 4.xlsx 94
Comparing Fractions 5_Grade 4.xlsx 67
Fraction as Number 1_Grade 4.xlsx 70
Fraction as Number 2_Grade 4.xlsx 108
Fraction as Number 3_Grade 4.xlsx 97
Fraction as Number 4_Grade 4.xlsx 47
Fraction as Number 5_Grade 4.xlsx 190
Fraction as Number 6_Grade 4.xlsx 111
Gang of Four_Grade 4.xlsx 983
HILLARY LEWIS-WOLFSEN pt 2.xlsx 44
HILLARY LEWIS-WOLFSEN pt 3.xlsx 63
HILLARY LEWIS-WOLFSEN pt 4.xlsx 66
HILLARY LEWIS-WOLFSEN pt 5.xlsx 34
HILLARY LEWIS-WOLFSEN pt 6_.xlsx 103
HILLARY LEWIS-WOLFSEN pt 7_.xlsx 54
HILLARY LEWIS-WOLFSEN pt1.xlsx 91
HILLARY LEWIS-WOLFSO

TalkBack.Year1.Reeves.Spring.022620.xlsx 618
TalkBack.Year1.Rossi.Fall.121919.xlsx 190
TalkBack.Year1.Rossi.Spring.011520.xlsx 583
TalkBack.Year1.Rossi.Spring.011920.xlsx 404
TalkBack.Year1.Rossi.Spring.012720.xlsx 641
TalkBack.Year1.Saunders.Spring.012320.xlsx 490
TalkBack.Year1.Saunders.Spring.012420.xlsx 583
TalkBack.Year1.Saunders.Spring.020320-1.xlsx 760
TalkBack.Year1.Saunders.Spring.020320-2.xlsx 538
TalkBack.Year1.Saunders.Spring.020520.xlsx 376
TalkBack.Year1.Saunders.Spring.020620-1.xlsx 254
TalkBack.Year1.Saunders.Spring.020620-2.xlsx 71
TalkBack.Year1.Saunders.Spring.020720.xlsx 219
TalkBack.Year1.Saunders.Spring.021020-1.xlsx 355
TalkBack.Year1.Saunders.Spring.021020-2.xlsx 416
TalkBack.Year1.Saunders.Spring.021820(1).xlsx 378
TalkBack.Year1.Saunders.Spring.021820.xlsx 935
TalkBack.Year1.Saunders.Spring.022020.xlsx 628
TalkBack.Year1.Smith.Fall.111919.xlsx 687
TalkBack.Year1.Smith.Fall.120319.xlsx 469
TalkBack.Year1.Smith.Fall.120919.xlsx 250
TalkBack.Year1.Smith.Spring.01

TalkBack.Year2.Parker.Fall.110920.xlsx 661
TalkBack.Year2.Parker.Fall.111020.xlsx 877
TalkBack.Year2.Parker.Fall.111320.xlsx 275
TalkBack.Year2.Parker.Spring.030921.xlsx 969
TalkBack.Year2.Parker.Spring.033121.xlsx 1005
TalkBack.Year2.Parker.Spring.041920.xlsx 794
TalkBack.Year2.Reeves.042921.xlsx 566
TalkBack.Year2.Smith.Fall.101920.xlsx 441
TalkBack.Year2.Smith.Fall.102220.xlsx 356
TalkBack.Year2.Smith.Fall.102320.xlsx 430
TalkBack.Year2.Smith.Fall.103020.xlsx 418
TalkBack.Year2.Smith.Fall.110520.xlsx 295
TalkBack.Year2.Smith.Fall.111020.xlsx 457
TalkBack.Year2.Smith.Fall.111220.xlsx 470
TalkBack.Year2.Smith.Fall.111920.xlsx 631
TalkBack.Year2.Smith.Spring.021721.xlsx 564
TalkBack.Year2.Smith.Spring.022221.xlsx 499
TalkBack.Year2.Smith.Spring.041921.xlsx 457
TalkBack.Year2.Smith.Spring.042121.xlsx 412
TalkBack.Year2.Smith.Spring.042221.xlsx 463
TalkBack.Year2.Smith.Spring.042821.xlsx 381
TalkBack.Year2.Smith.Spring.050421.xlsx 483
TalkBack.Year2.Smith.Spring.081321.xlsx 298
TalkBack.

In [116]:
temp_df = df[df['Speaker'] == 'T']
temp_df['Tag'].value_counts(), temp_df['Tag'].value_counts().sum()


(0    116973
 5     22925
 1     22776
 4      3998
 2      2862
 3      2613
 6      2039
 Name: Tag, dtype: int64,
 174186)

In [112]:
df['word_count'] = df['Sentence'].apply(lambda x: len(str(x).split(' ')))

In [113]:
df['word_count'].sum()

1792509

In [44]:
numberOfTurns, prevTurn = 0, ''
for item in df.iterrows():
    if prevTurn == '': 
        numberOfTurns += 1
        prevTurn = str(item[1]['Turn'])
    if str(item[1]['Turn']) != str(prevTurn):
        numberOfTurns += 1
        prevTurn = str(item[1]['Turn'])
print(numberOfTurns)

87071


In [114]:
words = df['Sentence'].str.lower().str.findall("\w+")

unique = set()

for x in words:
    unique.update(x)
print(len(words),len(unique))

234060 15830


In [15]:
df2 = df.loc[df['Speaker']=='T']
df2.groupby('Tag').agg(['count', 'size', 'nunique']).stack()

Turn  Speaker  Sentence  StudentTag  Transcript
Tag                                                           
0   count    105937   111836    111836      111836      111836
    size     111836   111836    111836      111836      111836
    nunique    1169        1     86866           1         534
1   count     20165    21400     21400       21400       21400
    size      21400    21400     21400       21400       21400
    nunique     953        1     16468           1         533
2   count      2504     2653      2653        2653        2653
    size       2653     2653      2653        2653        2653
    nunique     395        1      2267           1         382
3   count      2355     2458      2458        2458        2458
    size       2458     2458      2458        2458        2458
    nunique     453        1      1414           1         405
4   count      3618     3783      3783        3783        3783
    size       3783     3783      3783        3783        3783
    nunique     526        1      3686           1         456
5   count     20322    21627     21627       21627       21627
    size      21627    21627     21627       21627       21627
    nunique     814        1     19600           1         530
6   count      1794     1917      1917        1917        1917
    size       1917     1917      1917        1917        1917
    nunique     373        1      1518           1         368

In [32]:
writer = pd.ExcelWriter('../data/train_data_504.xlsx') #replace with test_data_63 for test data
df.to_excel(writer)
writer.save()

In [ ]:
'''
Pre-processing 

- All lower cased
- All punctuation removed
- Converted all teachers to T tags
- Converted all students to S tags
- T labels range from  [0-6] - 0 stands for None
- S are labelled None
- Compiled all the data into single excel sheet
- At present there are 91891 sentences in total

Cleaning

- [End of Class], [Class Ends] - Sentence in brackets
- If Sentence contains a timestamp or date time stamp
- If sentence is just . or ! or ? or empty
- Speaker is a number (All of them are nans)
- Speaker is nan


'''

In [33]:
#Prepare Tuples from data - Student
data = pd.read_excel('../data/train_data_504.xlsx') #replace with test_data_63 to generate test data
empty_previous = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        continue
    if row[1]['Speaker'] == 'S' and flag == 0:
        temp =  row[1]['Sentence']
        flag=1
        sentence_tuple.append([empty_previous,temp,row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['StudentTag']])
        y.append(row[1]['StudentTag'])
        temp =  row[1]['Sentence']
        continue
        
df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], 'text_b':[i[1] for i in sentence_tuple], 'labels':[i[2] for i in sentence_tuple]})
print(len(df),set(y))
df.to_csv('../data/train_student.tsv',sep='\t',index=False)   #replace with test_student to generate test data  

6905 {0.0, 1.0, 2.0, 3.0, 4.0}


In [34]:
#Prepare Tuples from data - teacher
data = pd.read_excel('../data/train_data_504.xlsx') #replace with test_data_63 to generate test data
empty_student = ''
sentence_tuple = []
y = []
flag  = 0
# T->T 0->1 T->S S->S S->T
for row in data.iterrows():
    if row[1]['Speaker'] == 'S' and flag == 2:
        temp =  row[1]['Sentence']
        continue
    if row[1]['Speaker'] == 'S' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 2
        continue
    if row[1]['Speaker'] == 'T' and flag == 0:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 1:
        temp =  row[1]['Sentence']
        flag = 1
        sentence_tuple.append([empty_student,temp,row[1]['Tag']])
        y.append(row[1]['Tag'])
        continue
    if row[1]['Speaker'] == 'T' and flag == 2:
        sentence_tuple.append([temp,row[1]['Sentence'],row[1]['Tag']])
        y.append(row[1]['Tag'])
        flag = 1
        continue

df = pd.DataFrame({'text_a':[i[0] for i in sentence_tuple], 'text_b':[i[1] for i in sentence_tuple], 'labels':[i[2] for i in sentence_tuple]})
print(len(df),set(y))
df.to_csv('../data/train_teacher.tsv',sep='\t',index=False)  #replace with test_teacher to generate test data 

21103 {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0}
